In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
def load_data():
    """
    Loads data from either source csv or from cleaned cache. Six (6) files
    are expected
        * data/arvato_data/Udacity_AZDIAS_052018.csv
        * data/arvato_data/Udacity_CUSTOMERS_052018.csv
        * data/arvato_data/Udacity_MAILOUT_052018_TEST.csv
        * data/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv
        
        * data/DIAS Attributes - Values 2017.xlsx
        * data/DIAS Information Levels - Attributes 2017.xlsx
    
    Returns:
        azdias, customer, mailout_test, mailout_train, attributes, information
    """
    def only_named_columns(col):
        """
        Removes columns from DataFrame while loaded that are unnamed
        """
        return 'Unnamed' not in col
    
    # Attributes and Information
    
    attributes = pd.read_excel('data/DIAS Attributes - Values 2017.xlsx', header=1, usecols=only_named_columns)
    attributes[['Attribute', 'Description']] = attributes[['Attribute', 'Description']].ffill()
    
    information = pd.read_excel('data/DIAS Information Levels - Attributes 2017.xlsx', header=1, usecols=only_named_columns)
    
    print('loading azdias data...')
    azdias = pd.read_csv('data/arvato_data/Udacity_AZDIAS_052018.csv', sep=';', usecols=only_named_columns)
    print('loading customers data...')
    customers = pd.read_csv('data/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', usecols=only_named_columns)
    print('loading mailout test data')
    mailout_test = pd.read_csv('data/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';', usecols=only_named_columns)
    print('loading mailout training data')
    mailout_train = pd.read_csv('data/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';', usecols=only_named_columns)
    
    print('Returning data in this order: azdias, customers, mailout_test, mailout_train, attributes, information')

    return azdias, customers, mailout_test, mailout_train, attributes, information

In [3]:
azdias, customers, mailout_test, mailout_train, attributes, information = load_data()

loading azdias data...


/home/patrick/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


loading customers data...
loading mailout test data
loading mailout training data
Returning data in this order: azdias, customers, mailout_test, mailout_train, attributes, information


In [4]:
def clean_dataframe(df, attributes, drop_cols, missingness_cols):
    def clean_columns_18_19(df):
        columns = ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015']
        df[columns] = df.loc[:, columns].replace(['X', 'XX'], np.nan)
        df[columns] = df.loc[:, columns].astype(float)
        return df
    def ost_west_encoder(df):
        encoder = {
            'O': 0,
            'W': 1
        }
        df['OST_WEST_KZ'] = df['OST_WEST_KZ'].map(encoder)
        return df
    def male_encoder(df):
        encoder = {
            2: 0,
            1: 1
        }
        df['ANREDE_KZ'] = df['ANREDE_KZ'].map(encoder)
        return df
    def one_hot_encode(df, col, drop_first=False):
        dummies = pd.get_dummies(df[col], drop_first=drop_first)
        df_list = [df.drop(col, axis=1), dummies]
        return pd.concat(df_list, axis=1)
    def lookup_unknown_val(col, attributes):
        try:
            mask = (attributes['Attribute'] == col) & (attributes['Meaning'].str.startswith('unknown'))
            unknown_val = attributes.loc[mask, 'Value']
            split_string = unknown_val.astype(str).str.cat(sep=',')
            
            return [int(x) for x in split_string.split(',')]
        except ValueError:
            return []
    def replace_unknowns_with_nan(df, attributes):
        for col in df.columns:
            df[col] = df[col].replace(lookup_unknown_val(col, attributes), np.nan)
        return df
    def drop_columns(df, cols):
        drop_cols = list(set(df.columns).intersection(set(cols)))
        return df.drop(drop_cols, axis=1)

    
    df = clean_columns_18_19(df)
    df = one_hot_encode(df, ['ANREDE_KZ'], True)
    df = one_hot_encode(df, ['CAMEO_DEU_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ'], False)
    df = replace_unknowns_with_nan(df, attributes)
    df = drop_columns(df, drop_cols) # missing columns in descriptions
    df = drop_columns(df, missingness_cols) # columns with high missing values
    
    return df

# def scale_dataframes(df_list):

In [5]:
def missing_percentages(df):
    missing = pd.DataFrame(df.isna().sum() / df.shape[0]).reset_index()
    missing.columns = ['Attribute', 'Missing']
    return missing
def remove_features_by_missingness(df, threshold=1):
    missing = missing_percentages(df)
    cols = missing.loc[missing['Missing'] > threshold, 'Attribute'].tolist()
    return cols

In [6]:
missingness_cols = remove_features_by_missingness(azdias, 0.2)

In [7]:
# columns that are to be dropped because of a lack of information in the documentation
drop_cols = ['KBA13_ANTG2',
             'D19_SCHUHE',
             'D19_REISEN',
             'D19_VERSI_ONLINE_DATUM',
             'ALTER_KIND3',
             'KBA13_ANTG1',
             'D19_BANKEN_REST',
             'STRUKTURTYP',
             'CJT_TYP_5',
             'D19_TELKO_REST',
             'D19_SONSTIGE',
             'EINGEZOGENAM_HH_JAHR',
             'D19_BEKLEIDUNG_REST',
             'D19_ENERGIE',
             'GEMEINDETYP',
             'D19_KOSMETIK',
             'D19_WEIN_FEINKOST',
             'ALTER_KIND4',
             'KBA13_HHZ',
             'LNR',
             'D19_SAMMELARTIKEL',
             'D19_BANKEN_DIREKT',
             'ANZ_KINDER',
             'D19_LOTTO',
             'DSL_FLAG',
             'UNGLEICHENN_FLAG',
             'D19_VERSI_DATUM',
             'D19_RATGEBER',
             'D19_GARTEN',
             'D19_BUCH_CD',
             'RT_SCHNAEPPCHEN',
             'RT_UEBERGROESSE',
             'D19_BEKLEIDUNG_GEH',
             'KBA13_BAUMAX',
             'D19_BANKEN_LOKAL',
             'ALTER_KIND1',
             'D19_VERSI_ONLINE_QUOTE_12',
             'VHN',
             'D19_LEBENSMITTEL',
             'VK_ZG11',
             'D19_HANDWERK',
             'VERDICHTUNGSRAUM',
             'KOMBIALTER',
             'KONSUMZELLE',
             'D19_VOLLSORTIMENT',
             'CJT_TYP_4',
             'D19_TIERARTIKEL',
             'D19_VERSAND_REST',
             'D19_BIO_OEKO',
             'MOBI_RASTER',
             'UMFELD_ALT',
             'KBA13_ANTG4',
             'CJT_KATALOGNUTZER',
             'D19_NAHRUNGSERGAENZUNG',
             'CJT_TYP_2',
             'KBA13_KMH_210',
             'SOHO_KZ',
             'EINGEFUEGT_AM',
             'D19_TELKO_ONLINE_QUOTE_12',
             'CJT_TYP_3',
             'CJT_TYP_1',
             'ARBEIT',
             'D19_BANKEN_GROSS',
             'AKT_DAT_KL',
             'KBA13_GBZ',
             'VK_DISTANZ',
             'UMFELD_JUNG',
             'ANZ_STATISTISCHE_HAUSHALTE',
             'FIRMENDICHTE',
             'CJT_TYP_6',
             'D19_DROGERIEARTIKEL',
             'ALTER_KIND2',
             'D19_BILDUNG',
             'D19_DIGIT_SERV',
             'D19_KINDERARTIKEL',
             'HH_DELTA_FLAG',
             'EXTSEL992',
             'D19_TECHNIK',
             'RT_KEIN_ANREIZ',
             'D19_SOZIALES',
             'D19_TELKO_MOBILE',
             'KBA13_CCM_1401_2500',
             'ALTERSKATEGORIE_FEIN',
             'D19_FREIZEIT',
             'VK_DHT4A',
             'KBA13_ANTG3',
             'D19_KONSUMTYP_MAX',
             'VHA',
             'D19_VERSI_OFFLINE_DATUM',
             'D19_VERSICHERUNGEN',
             'KK_KUNDENTYP',
             'CAMEO_INTL_2015',
             'D19_HAUS_DEKO']

In [8]:
customer_specific_columns = ['ONLINE_PURCHASE', 'PRODUCT_GROUP', 'CUSTOMER_GROUP']
customer_specific_col_df = customers[customer_specific_columns]
customer_similar_col_df = customers.drop(customer_specific_columns, axis=1)

In [9]:
cleaned_customer = clean_dataframe(customer_similar_col_df, attributes, drop_cols, missingness_cols)

In [10]:
cleaned_azdias = clean_dataframe(azdias, attributes, drop_cols, missingness_cols)

In [11]:
def return_scaled_columns_list(df):
    scaled, not_scaled = df.select_dtypes(exclude='uint8').columns.tolist(), df.select_dtypes(include='uint8').columns.tolist()

    return scaled, not_scaled

def create_scaler(df):
    scaled_col_list, non_scaled_col_list = return_scaled_columns_list(df)
    
    scaler = StandardScaler()
    scaler.fit(df[scaled_col_list])
    
    return scaler
def scale_dataframe(df, scaler):
    scaled_col_list, non_scaled_col_list = return_scaled_columns_list(df)
    scaled_df = pd.DataFrame(scaler.transform(df[scaled_col_list]), columns=scaled_col_list)
    
    return pd.concat([scaled_df, df[non_scaled_col_list]], axis=1)

In [12]:
scaler = create_scaler(cleaned_azdias)

In [13]:
azdias_scaled = scale_dataframe(cleaned_azdias, scaler)

In [14]:
customer_scaled = scale_dataframe(cleaned_customer, scaler)

In [16]:
def create_imputer(df):
    imputer = SimpleImputer(strategy='median')
    imputer.fit(df)
    return imputer

In [17]:
def impute_values(df, imputer):
    return pd.DataFrame(imputer.transform(df), columns=df.columns)

In [18]:
imputer = create_imputer(azdias_scaled)

In [19]:
azdias_imputed = impute_values(azdias_scaled, imputer)

In [20]:
customer_imputed = impute_values(customer_scaled, imputer)